# **IA353 - Redes Neurais: EFC2 - Questão 3 e Questão 4**
## **Síntese de Modelos Baseados em Redes MLP e Convolucional para Classificação de Padrões**

**Professor:** Fernando J. Von Zuben.

**Aluno(a)**: Guilherme Rosa

### **1. Importações**

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

### **2. Download da base de dados MNIST**

  - A base de dados MNIST é constituída pelos conjuntos de treinamento e teste. O primeiro conjunto possui 60.000 amostras de imagens de dígitos manuscritos, enquanto que o segundo conjunto possui 10.000 amostras.
  - As amostras da base MNIST, quando baixadas utilizando a API do Keras, apresentam as seguintes características:
    - As entradas estão no formato (28, 28, 1), isto é, uma matriz quadrada com cada pixel no intervalo [0, 255].
    - As saídas (rótulos) estão na representação categórica.
  - Após o download, os pixels das amostras de entrada passam por um processo de normalização, sendo divididos por 255, de modo que seus valores fiquem no intervalo [0, 1].
  - Nesta atividade, a normalização é necessária para que o mapeamento a ser aproximado pelos modelos neurais seja mais suave, facilitando o processo de treinamento (i. e., a obtenção dos parâmetros do modelo).
  - As informações referentes ao formato (shape) dos dados de entrada é apresentado nas seções de implementação das redes neurais MLP e convolucional.
  - Diferente do que foi feito nas Questões 1 e 2 do EFC1, as saídas não são convertidas para a representação one-hot, pois a função custo **sparse_categorial_crossentropy** converte implicitamente as saídas categóricas para one-hot durante o treinamento.

In [2]:
(X_train, y), (X_test, y_test) = keras.datasets.mnist.load_data()

print('Valores máximo e mínimo antes da normalização:')
print(f'  X_train.max: {X_train.max()}')
print(f'  X_train.min: {X_train.min()}')

X_train, X_test = X_train/255., X_test/255.
print('Valores máximo e mínimo após a normalização:')
print(f'  X_train.max: {X_train.max()}')
print(f'  X_train.min: {X_train.min()}')

11493376/11490434 [==============================] - 0s 0us/step
Valores máximo e mínimo antes da normalização:
  X_train.max: 255
  X_train.min: 0
Valores máximo e mínimo após a normalização:
  X_train.max: 1.0
  X_train.min: 0.0


### **3. Questão 3: Classificador com Rede Neural MLP**

#### **3.1. Observações:**

- As entradas X_train e X_test devem ser convertidas de matriz (28, 28, 1) para vetor (784,) pois as redes neurais com arquitetura fully-connected apenas processam vetores.
  
- O objetivo do exercício é superar o desempenho da rede MLP proposta no enunciado, cujas especificações e desempenho estão apresentadas na Sub-seção 3.2.

- O desempenho utilizado para comparação será um desempenho médio tomado a partir de 5 realizações de uma dada configuração de hiperparâmetros. **A métrica de decisão será a acurácia junto aos dados de validação**.

- Os modelos utilizam o Dropout como técnica de regularização. Essa técnica desativa aleatoriamente uma parcela dos neurônios durante o treinamento. Após o treinamento, os pesos de cada neurônio é multiplicada pela probabilidade dele pertencer ao modelo. Assim, para que o modelo resultante seja obtido, **é NECESSÁRIO utilizar a método evaluate**.

- Os dados de treinamento são divididos em amostras de treinamento (80%) e amostras de validação (20%).

- Mudança do formato dos dados de entrada e separação dos dados em amostras de treinamento e de validação:

In [3]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]*X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]*X_test.shape[2]))

N = int(0.8*X_train.shape[0])

X_valid, y_valid = X_train[N:], y[N:]
X_train, y_train = X_train[:N], y[:N]

In [4]:
print('Dados de treinamento:')
print(f'  Dimensão de X_train: {X_train.shape}')
print(f'  Dimensão de y_train: {y_train.shape}')

Dados de treinamento:
  Dimensão de X_train: (48000, 784)
  Dimensão de y_train: (48000,)


In [5]:
print('Dados de validação:')
print(f'  Dimensão de X_valid: {X_valid.shape}')
print(f'  Dimensão de y_valid: {y_valid.shape}')

Dados de validação:
  Dimensão de X_valid: (12000, 784)
  Dimensão de y_valid: (12000,)


In [6]:
print('Dados de teste:')
print(f'  Dimensão de X_test: {X_test.shape}')
print(f'  Dimensão de y_test: {y_test.shape}')

Dados de teste:
  Dimensão de X_test: (10000, 784)
  Dimensão de y_test: (10000,)


#### **3.2. Arquitetura proposta no enunciado:**

  - Uma camada intermediária com 512 neurônios com funções de ativação ReLU e taxa de ocorrência de *dropout* de 50%.
  - Uma camada de saída com 10 neurônios com função de ativação softmax (não deve ser alterada).
  - Algoritmo de otimização Adam, aplicado durante 5 épocas e com 32 amostras por mini-batch.
  - Função custo (loss): sparse_categorical_crossentropy (não deve ser alterada).

In [7]:
N = 5 # número de realizações

metrics = []
print(f'Treinamento do modelo proposto:', end=' ')

for i in range(1, N+1):

    mlp_enunciado = keras.models.Sequential([
        keras.layers.Input(shape=(784, )),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])

    mlp_enunciado.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    mlp_enunciado.fit(X_train, y_train, batch_size=32, validation_data=(X_valid, y_valid), epochs=5, verbose=False)

    loss_t, acc_t = mlp_enunciado.evaluate(X_train, y_train, verbose=False)
    loss_v, acc_v = mlp_enunciado.evaluate(X_valid, y_valid, verbose=False)
    metrics.append((loss_t, acc_t, loss_v, acc_v))

print(f'Finalizado')

Treinamento do modelo proposto: Finalizado


In [8]:
metrics = np.array(metrics)
metrics_mean = np.reshape(np.mean(metrics, axis=0), (1, 4))

df = pd.DataFrame(metrics_mean, columns=['Train_loss', 'Train_acc', 'Val_loss', 'Val_acc'], index=['Modelo proposto'])
df

,Train_loss,Train_acc,Val_loss,Val_acc
Modelo proposto,0.036767,0.988808,0.079167,0.977183


#### **3.3. Análise do impacto de alguns hiperparâmetros da primeira camada intermediária no desempenho da rede MLP:**

#####**3.3.1. Impacto de diferentes funções de ativação:**

In [9]:
activations = ['sigmoid', 'tanh', 'relu', 'leakyrelu']

activation_metrics = {}

for ACTIVATION in activations:
    print(f'Treinamento do modelo com funções de ativação do tipo {ACTIVATION}:', end=' ')
    
    metrics = []    
    for i in range(1, N+1):

        if ACTIVATION != 'leakyrelu':
            mlp = keras.models.Sequential([
                keras.layers.Input(shape=(784, )),
                keras.layers.Dense(512, activation=ACTIVATION),
                keras.layers.Dropout(0.5),
                keras.layers.Dense(10, activation='softmax')
            ])
        else:
           mlp = keras.models.Sequential([
                keras.layers.Input(shape=(784, )),
                keras.layers.Dense(512),
                keras.layers.LeakyReLU(),
                keras.layers.Dropout(0.5),
                keras.layers.Dense(10, activation='softmax')
            ])
    
        mlp.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        mlp.fit(X_train, y_train, batch_size=32, validation_data=(X_valid, y_valid), epochs=5, verbose=False)

        loss_t, acc_t = mlp.evaluate(X_train, y_train, verbose=False)
        loss_v, acc_v = mlp.evaluate(X_valid, y_valid, verbose=False)
        metrics.append((loss_t, acc_t, loss_v, acc_v))

    print('Finalizado')
    metrics = np.array(metrics)
    metrics_mean = np.mean(metrics, axis=0)
    activation_metrics[ACTIVATION] = metrics_mean

Treinamento do modelo com funções de ativação do tipo sigmoid: Finalizado
Treinamento do modelo com funções de ativação do tipo tanh: Finalizado
Treinamento do modelo com funções de ativação do tipo relu: Finalizado
Treinamento do modelo com funções de ativação do tipo leakyrelu: Finalizado


In [10]:
df = pd.DataFrame(activation_metrics.values(), columns=['Train_loss', 'Train_acc', 'Val_loss', 'Val_acc'], index=activation_metrics.keys())
df

,Train_loss,Train_acc,Val_loss,Val_acc
sigmoid,0.077281,0.977642,0.104218,0.968917
tanh,0.076533,0.976508,0.114284,0.966800
relu,0.038913,0.988083,0.083123,0.975650
leakyrelu,0.102350,0.969229,0.139170,0.962217


In [11]:
val_accs = [metric[-1] for metric in list(activation_metrics.values())]            # Lista que contém apenas as acurácias de validação
activations_val_accs = list(zip(val_accs, list(activation_metrics.keys())))        # Lista que contém o par Função de ativação - Acurácia de validação

activations_val_accs.sort(reverse=True)                                            # Ordena as acurácias do maior para o menor
best_activation = activations_val_accs[0][1]
print(f'Função de ativação que levou ao melhor desempenho: {best_activation}')

Função de ativação que levou ao melhor desempenho: relu


- Como mostra a tabela acima, o uso de funções de ativação sigmoidais (sigmoide e tangente hiperbólica) levaram a modelos com desempenho inferior ao uso da ReLU.

- É interessante destacar que neste problema de classificação de dígitos manuscritos, a LeakyReLU também apresentou desempenho médio inferior ao da ReLU.

##### **3.3.2. Impacto de diferentes algoritmos de otimização:**

In [12]:
optimizers = ['sgd', 'sgd+momentum', 'NAG', 'adagrad', 'adadelta', 'adam']

optimizers_metrics = {}

for OPTIMIZER in optimizers:
    print(f'Treinamento do modelo com algoritmo de otimização {OPTIMIZER}:', end=' ')
    
    metrics = []    
    for i in range(1, N+1):
        mlp = keras.models.Sequential([
            keras.layers.Input(shape=(784, )),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10, activation='softmax')
        ])

        if OPTIMIZER == 'sgd+momentum':
            opt = keras.optimizers.SGD(momentum=0.9)
            mlp.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        elif OPTIMIZER == 'NAG':
            opt = keras.optimizers.SGD(momentum=0.9, nesterov=True)
            mlp.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        else:
            mlp.compile(loss='sparse_categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])
        
        mlp.fit(X_train, y_train, batch_size=32, validation_data=(X_valid, y_valid), epochs=5, verbose=False)

        loss_t, acc_t = mlp.evaluate(X_train, y_train, verbose=False)
        loss_v, acc_v = mlp.evaluate(X_valid, y_valid, verbose=False)
        metrics.append((loss_t, acc_t, loss_v, acc_v))

    print('Finalizado')
    metrics = np.array(metrics)
    metrics_mean = np.mean(metrics, axis=0)
    optimizers_metrics[OPTIMIZER] = metrics_mean

Treinamento do modelo com algoritmo de otimização sgd: Finalizado
Treinamento do modelo com algoritmo de otimização sgd+momentum: Finalizado
Treinamento do modelo com algoritmo de otimização NAG: Finalizado
Treinamento do modelo com algoritmo de otimização adagrad: Finalizado
Treinamento do modelo com algoritmo de otimização adadelta: Finalizado
Treinamento do modelo com algoritmo de otimização adam: Finalizado


In [13]:
df = pd.DataFrame(optimizers_metrics.values(), columns=['Train_loss', 'Train_acc', 'Val_loss', 'Val_acc'], index=optimizers_metrics.keys())
df

,Train_loss,Train_acc,Val_loss,Val_acc
sgd,0.224920,0.937496,0.218798,0.939967
sgd+momentum,0.059615,0.983237,0.086807,0.974450
NAG,0.060603,0.982892,0.088125,0.974317
adagrad,0.402463,0.896254,0.375811,0.904800
adadelta,1.504049,0.731687,1.484198,0.750217
adam,0.036506,0.988792,0.078927,0.976600


In [14]:
val_accs = [metric[-1] for metric in list(optimizers_metrics.values())]            # Lista que contém apenas as acurácias de validação
optimizers_val_accs = list(zip(val_accs, list(optimizers_metrics.keys())))         # Lista que contém o par Algoritmo de Otimização - Acurácia de validação

optimizers_val_accs.sort(reverse=True)                                             # Ordena as acurácias do maior para o menor
best_optimizer = optimizers_val_accs[0][1]
print(f'Algoritmo de otimização que levou ao melhor desempenho: {best_optimizer}')

Algoritmo de otimização que levou ao melhor desempenho: adam


- Conforme mostra a tabela acima, os algoritmos que levaram à redes com os piores desempenhos médios foram o SGD, Adagrad e Adadelta.

- No caso do SGD, esse desempenho insatisfatório deve-se ao algoritmo não apresentar um passo adaptativo, isto é, o valor fixo para o passo de ajuste dos parâmetros deve ter sido muito pequeno, a ponto de convergir para um mínimo local ruim.

- No caso do Adagrad, apresentou um desempenho ruim mesmo sendo um algoritmo adaptativo. Isso ocorre pois a cada iteração os passos de ajuste dos pesos decrescem monotonicamente, levando-o a convergir para um mínimo ainda pior que o do SGD. 

- Por fim, observa-se que o Adadelta se mostrou como o pior algoritmo. Em teoria ele deveria levar a um desempenho superior ao do Adagrad, pois sua formulação combate o problema de decrescimento monotônico.

- Por outro lado, os algoritmos que levaram à redes com os melhores desempenhos médios foram SGD+momentum, NAG e Adam, sendo este último o melhor.

##### **3.3.3. Impacto do número de neurônios:**

In [15]:
units = [100, 300, 512, 700, 900]

units_metrics = {}

for NUM_UNITS in units:
    print(f'Treinamento do modelo com {NUM_UNITS} neurônios na camada intermediária:', end=' ')
    
    metrics = []    
    for i in range(1, N+1):
        mlp = keras.models.Sequential([
            keras.layers.Input(shape=(784, )),
            keras.layers.Dense(NUM_UNITS, activation='relu'),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10, activation='softmax')
        ])
        
        mlp.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        mlp.fit(X_train, y_train, batch_size=32, validation_data=(X_valid, y_valid), epochs=5, verbose=False)

        loss_t, acc_t = mlp.evaluate(X_train, y_train, verbose=False)
        loss_v, acc_v = mlp.evaluate(X_valid, y_valid, verbose=False)
        metrics.append((loss_t, acc_t, loss_v, acc_v))

    print('Finalizado')
    metrics = np.array(metrics)
    metrics_mean = np.mean(metrics, axis=0)
    units_metrics[str(NUM_UNITS)] = metrics_mean

Treinamento do modelo com 100 neurônios na camada intermediária: Finalizado
Treinamento do modelo com 300 neurônios na camada intermediária: Finalizado
Treinamento do modelo com 512 neurônios na camada intermediária: Finalizado
Treinamento do modelo com 700 neurônios na camada intermediária: Finalizado
Treinamento do modelo com 900 neurônios na camada intermediária: Finalizado


In [16]:
df = pd.DataFrame(units_metrics.values(), columns=['Train_loss', 'Train_acc', 'Val_loss', 'Val_acc'], index=units_metrics.keys())
df

,Train_loss,Train_acc,Val_loss,Val_acc
100,0.085686,0.974825,0.111537,0.968167
300,0.044490,0.986642,0.083971,0.975133
512,0.037403,0.988442,0.081164,0.976017
700,0.037202,0.988133,0.083478,0.975317
900,0.033207,0.989600,0.080359,0.976617


In [17]:
val_accs = [metric[-1] for metric in list(units_metrics.values())]      # Lista que contém apenas as acurácias de validação
units_val_accs = list(zip(val_accs, list(units_metrics.keys())))        # Lista que contém o par Número de neurônios - Acurácia de validação

units_val_accs.sort(reverse=True)                                       # Ordena as acurácias do maior para o menor
best_units = units_val_accs[0][1]
print(f'Número de neurônios que levou ao melhor desempenho: {best_units}')

Número de neurônios que levou ao melhor desempenho: 900


- Conforme mostra a tabela acima, a rede MLP com 900 neurônios na camada intermediária apresentou um desempenho médio levemente superior ao da arquitetura proposta com 512 neurônios.

- Isso nos leva a concluir que o aumento da flexibilidade do modelo com a inserção de mais neurônios na camada intermediária não leva a uma melhora significativa do desempenho.

##### **3.3.4. Impacto da taxa de ocorrência de dropout:**

- Como as técnicas de regularização dependem da flexibilidade do modelo, não faria sentido analisar o impacto da taxa de ocorrência de Dropout considerando os 512 neurônios na camada intermediária da arquitetura proposta e, em seguida, criar um modelo que combine todos os melhores hiperparâmetros obtidos *best_activation*, *best_optimizer*, *best_units* e *best_rate*.

- Diante disso, a análise do impacto da taxa de ocorrência de Dropout é feita considerando os melhores valores para os 3 hiperparâmetros já analisados *best_activation*, *best_optimizer*, *best_units*.

In [23]:
dropout_rates = [0.1, 0.3, 0.5, 0.7, 0.9]

rates_metrics = {}

for DROPOUT_RATE in dropout_rates:
    print(f'Treinamento do modelo com taxa de dropout de {DROPOUT_RATE*100}%:', end=' ')
    
    metrics = []    
    for i in range(1, N+1):
        mlp = keras.models.Sequential([
            keras.layers.Input(shape=(784, )),
            keras.layers.Dense(best_units, activation=best_activation),
            keras.layers.Dropout(DROPOUT_RATE),
            keras.layers.Dense(10, activation='softmax')
        ])
        
        mlp.compile(loss='sparse_categorical_crossentropy', optimizer=best_optimizer, metrics=['accuracy'])
        mlp.fit(X_train, y_train, batch_size=32, validation_data=(X_valid, y_valid), epochs=5, verbose=False)

        loss_t, acc_t = mlp.evaluate(X_train, y_train, verbose=False)
        loss_v, acc_v = mlp.evaluate(X_valid, y_valid, verbose=False)
        metrics.append((loss_t, acc_t, loss_v, acc_v))

    print('Finalizado')
    metrics = np.array(metrics)
    metrics_mean = np.mean(metrics, axis=0)
    rates_metrics[DROPOUT_RATE] = metrics_mean

Treinamento do modelo com taxa de dropout de 10.0%: Finalizado
Treinamento do modelo com taxa de dropout de 30.0%: Finalizado
Treinamento do modelo com taxa de dropout de 50.0%: Finalizado
Treinamento do modelo com taxa de dropout de 70.0%: Finalizado
Treinamento do modelo com taxa de dropout de 90.0%: Finalizado


In [24]:
df = pd.DataFrame(rates_metrics.values(), columns=['Train_loss', 'Train_acc', 'Val_loss', 'Val_acc'], index=rates_metrics.keys())
df

,Train_loss,Train_acc,Val_loss,Val_acc
0.1,0.023935,0.992175,0.090907,0.975483
0.3,0.025981,0.991717,0.084010,0.976817
0.5,0.031324,0.990192,0.079429,0.978233
0.7,0.051205,0.984033,0.088202,0.974667
0.9,0.118692,0.965154,0.130039,0.963000


In [25]:
val_accs = [metric[-1] for metric in list(rates_metrics.values())]      # Lista que contém apenas as acurácias de validação
rates_val_accs = list(zip(val_accs, list(rates_metrics.keys())))        # Lista que contém o par Taxa de Dropout - Acurácia de validação

rates_val_accs.sort(reverse=True)                                       # Ordena as acurácias do maior para o menor
best_rate = rates_val_accs[0][1]
print(f'Taxa de Dropout que levou ao melhor desempenho: {best_rate}')

Taxa de Dropout que levou ao melhor desempenho: 0.5


##### **3.3.5. Discussão**

- Vimos na seção 3.3 como o desempenho médio da rede neural MLP com uma única camada intermediária é afetada com a variação da função de ativação, algoritmo de otimização, número de neurônios da camada oculta e taxa de dropout.

- Na subseção 3.3.4 foram combinados os melhores valores de hiperparâmetros. Contudo, a arquitetura resultante não levou a um aumento significativo da acurácia junto aos dados de validação.

- Com isso, somos motivados a estudar redes MLP com mais camadas intermediárias e investigar se um aumento maior no desempenho médio é alcançado. 

#### **3.4. Análise do impacto da inserção de uma segunda camada intermediária no desempenho da rede MLP:**

#### **3.5. Arquitetura com desempenho superior:**

- Na sub-seção 3.3.4 foi obtido uma arquitetura com desempenho médio superior ao da arquitetura proposta no enunciado.

- Hiperparâmetros:


- A seguir, ambas as arquiteturas são retreinadas e os desempenhos médios são tomados a partir de 10 realizações.

In [30]:
N = 10 # número de realizações

metrics_mlp_enunciado = []
print(f'Treinamento do modelo proposto:', end=' ')

for i in range(1, N+1):

    mlp_enunciado = keras.models.Sequential([
        keras.layers.Input(shape=(784, )),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])

    mlp_enunciado.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    mlp_enunciado.fit(X_train, y_train, batch_size=32, validation_data=(X_valid, y_valid), epochs=5, verbose=False)

    loss_t, acc_t = mlp_enunciado.evaluate(X_train, y_train, verbose=False)
    loss_v, acc_v = mlp_enunciado.evaluate(X_valid, y_valid, verbose=False)
    metrics_mlp_enunciado.append((loss_t, acc_t, loss_v, acc_v))

print(f'Finalizado')

Treinamento do modelo proposto: Finalizado


In [31]:
metrics_mlp_alternativo = []
print(f'Treinamento do modelo alternativo:', end=' ')

for i in range(1, N+1):

    mlp_alternativo = keras.models.Sequential([
          # Modelo
    ])

    mlp_alternativo.compile(loss='sparse_categorical_crossentropy', optimizer=best_optimizer, metrics=['accuracy'])
    mlp_alternativo.fit(X_train, y_train, batch_size=32, validation_data=(X_valid, y_valid), epochs=5, verbose=False)

    loss_t, acc_t = mlp_alternativo.evaluate(X_train, y_train, verbose=False)
    loss_v, acc_v = mlp_alternativo.evaluate(X_valid, y_valid, verbose=False)
    metrics_mlp_alternativo.append((loss_t, acc_t, loss_v, acc_v))

print(f'Finalizado')

Treinamento do modelo alternativo: Finalizado


In [32]:
metrics_mlp_enunciado = np.array(metrics_mlp_enunciado)
metrics_mlp_enunciado_mean = np.reshape(np.mean(metrics_mlp_enunciado, axis=0), (1, 4))

metrics_mlp_alternativo = np.array(metrics_mlp_alternativo)
metrics_mlp_alternativo_mean = np.reshape(np.mean(metrics_mlp_alternativo, axis=0), (1, 4))

metrics_mean = metrics_mlp_enunciado_mean.copy()
metrics_mean = np.append(metrics_mean, metrics_mlp_alternativo_mean, axis=0)

df = pd.DataFrame(metrics_mean, columns=['Train_loss', 'Train_acc', 'Val_loss', 'Val_acc'], index=['Modelo proposto', 'Modelo alternativo'])
df

,Train_loss,Train_acc,Val_loss,Val_acc
Modelo proposto,0.037401,0.988433,0.080994,0.976217
Modelo alternativo,0.034220,0.989327,0.082900,0.976150


# **Exercício de Fixação de Conceitos 2 - Questão 4**

### **Enunciado**:
- Tomando o mesmo problema de classificação de dados da base MNIST e novamente usando o *framework* Keras, tendo o TensorFlow como *backend*, realize o treinamento de uma rede neural com camadas convolucionais, usando *maxpooling* e *dropout*.
- Mais uma vez, é apresentada a seguir uma sugestão de código e de configuração de hiperparâmetros que pode ser tomada como ponto de partida.
- A sua proposta deve superar, em termos de desempenho médio, essa sugestão fornecida abaixo.
- Descreva de forma objetiva o caminho trilhado até sua configuração final de código.
- Compare os resultados (em termos de taxa de acerto de classificação) com aqueles obtidos pelos três tipos de máquinas de aprendizado adotadas nas atividades anteriores (classificador linear, ELM e MLP).


In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

### **Parte 1 - Solução do exercício:**
- Diferente do que foi feito na questão 2, onde treinamos 5 redes neurais para cada configuração de hiperparâmetros, aqui são treinadas apenas 3 redes, pois o tempo de treinamento de uma rede convolucional é muito maior que de uma simples rede MLP.

**a) Arquitetura e treinamento propostos no enunciado:**
- Arquitetura:
  - Uma camada convolucional com 32 kernels de dimensão 3x3, stride (1,1) e com funções de ativação ReLU.
  - Uma segunda camada convolucional com 64 kernels de dimensão 3x3, stride (1,1) e com funções de ativação ReLU, MaxPooling de dimensão 2x2 e taxa de ocorrência de dropout de 25%.
  - Camada do tipo Flatten
  - Camada fully-connected com 128 neurônios com função de ativação ReLU e taxa de ocorrência de dropout de 50%.
  - Uma camada de saída com 10 neurônios com função de ativação softmax (não deve ser alterada).
- Treinamento: 
  - Algoritmo adaptativo Adam, 5 épocas e 32 amostras por mini-batch (default do método fit()).
  - Função custo (loss): sparse_categorical_crossentropy (não deve ser alterada).
  - Métrica auxiliar: Acurácia (não deve ser alterada).
- Resultado do treinamento dos 3 modelos:
  - Perdas: 0.0434
  - Acurácia: 0.9866

**b) Alterações na rede convolucional proposta e seus impactos no desempenho:**

- A primeira etapa das alterações foi relacionada aos hiperparâmetros da primeira camada convolucional. Foram realizadas as seguintes modificações:
  - 1) Redução de 32 para 16 kernels
  - 2) Aumento de 32 para 64 kernels
  - 3) Redução do tamanho dos kernels de 3x3 para 2x2
  - 4) Aumento do tamanho dos kernels de 3x3 para 4x4
  - 5) Aumento do stride de (1, 1) para (2, 2)
  - 6) Inserção de uma camada de Max Pooling
- A Tabela 5 apresenta as métricas de desempenho, perdas e acurácia, para as redes com cada uma das alterações acima.

<h><center>Tabela 5: Métricas de desempenho para cada uma das alterações de hiperparâmetros realizadas na primeira camada convolucional.</center></h>

| Alteração | Perdas | Acurácia |
|-----------------|--------|----------|
|1              |0.0421  |0.9871    |
|2              |0.0439  |0.9864    |
|3              |0.0464  |0.9855    |
|4              |0.0409  |0.9873    |
|5              |0.0489  |0.9850    |
|6              |0.0519  |0.9841    |

- Como pode ser observado na Tabela 5, duas alterações levaram a um aumento no desempenho: redução do número de kernels de 32 para 16 e o aumento das dimensões de cada kernel de 3x3 para 4x4.
- Diante disso, foram treinadas redes com essas duas alterações juntas visando alcançar um desempenho ainda maior. O resultado dessa investigação está apresentado na Tabela 6. Nota-se que o desempenho dessa configuração foi menor do que o desempenho das redes com cada uma das alterações feitas individualmente.
- A segunda parte das alterações foram feitas considerando a primeira camada convolucional com 32 kernels de dimensão 4x4. As modificações, cujo os desempenhos estão apresentados na Tabela 6, foram:
  - 7) Remoção da camada de Max Pooling da segunda camada convolucional
  - 8) Remoção da camada fully-connected.  

<h><center>Tabela 6: Métricas de desempenho para a segunda parte de alterações realizadas na rede convolucional.</center></h>

| Alteração | Perdas | Acurácia |
|---------------|--------|----------|
|1+4            |0.0430  |0.9869    |
|7              |0.0342  |0.9894    |
|8              |0.0239  |0.9924    |
|7+8            |0.0158  |0.9949    |

- Pode-se observar que as modificações 7 e 8 levaram a um aumento substancial no desempenho da rede, atingindo uma acurácia de 99,49% com as duas alterações feitas conjuntamente.


**c) Arquitetura final que supera o desempenho da rede proposta no enunciado:**
- Arquitetura:
  - Uma camada convolucional com 32 kernels de dimensão 4x4, stride (1,1) e com funções de ativação ReLU.
  - Uma segunda camada convolucional com 64 kernels de dimensão 3x3, stride (1,1), com funções de ativação ReLU e taxa de ocorrência de dropout de 25%.
  - Camada do tipo Flatten
  - Uma camada de saída com 10 neurônios com função de ativação softmax (não deve ser alterada).
- Treinamento: 
  - Algoritmo adaptativo Adam, 5 épocas e 32 amostras por mini-batch (default do método fit()).
  - Função custo (loss): sparse_categorical_crossentropy (não deve ser alterada).
  - Métrica auxiliar: Acurácia (não deve ser alterada).
- Resultado do treinamento dos 3 modelos:
  - Perdas: 0.0158
  - Acurácia: 0.9949

In [ ]:
# Rede proposta: 32 kernels na camada 1
num_models = 3
EPOCHS = 5
media_metricas = []

lista_metricas = { 'loss': [], 'accuracy': []}
metricas = {}

for i in range(0, num_models):
    CNN = keras.models.Sequential([
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[1:]),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Dropout(0.25),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])

    CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = CNN.fit(x_train, y_train, epochs=5)

    lista_metricas['loss'].append(history.history['loss'][-1])
    lista_metricas['accuracy'].append(history.history['accuracy'][-1])

for key in lista_metricas.keys():
    metricas[key] = sum(lista_metricas[key])/len(lista_metricas[key])

media_metricas.append(metricas)
print(media_metricas)

In [ ]:
# Rede final:
num_models = 3
EPOCHS = 5
media_metricas = []

lista_metricas = { 'loss': [], 'accuracy': []}
metricas = {}

for i in range(0, num_models):
    CNN = keras.models.Sequential([
        keras.layers.Conv2D(32, kernel_size=(4, 4), activation='relu', input_shape=x_train.shape[1:]),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        keras.layers.Dropout(0.25),
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation='softmax')
    ])

    CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = CNN.fit(x_train, y_train, epochs=5)

    lista_metricas['loss'].append(history.history['loss'][-1])
    lista_metricas['accuracy'].append(history.history['accuracy'][-1])

for key in lista_metricas.keys():
    metricas[key] = sum(lista_metricas[key])/len(lista_metricas[key])

media_metricas.append(metricas)
print(media_metricas)

### **Parte 2 - Comparação de todos os modelos:**

- A Tabela 7 apresenta os desempenhos dos 4 classificadores estudados nas questões 1 a 4.
- Classificador Linear:
  - Coeficiente de Regularização: 965.8832
  - Critério de quadrados mínimos
- Máquina de Aprendizado Extremo:
  - Camada intermediária: 1000 neurônios com função de ativação ReLU com pesos definidos aleatoriamente de acordo com uma função de distribuição normal com média nula e desvio padrão de 0.2.
  - Camada de saída: 10 neurônios com função de ativação linear.
  - Critério de quadrados mínimos
- Redes MLP e convolucional: estruturas já apresentados neste *notebook* (item f da questão 3 e item c da questão 4, respectivamente).  

<h><center>Tabela 7: Desempenho dos 4 modelos de classificadores obtidos nos exercícios dos EFCs 1 e 2 junto aos dados de treinamento.</center></h>

| Modelo de Classificador  | Acurácia | Parâmetros ajustáveis |
|-------------------------------|----------|------------------|
|Linear                         |0.8570    |  7850            |
|Máquina de Aprendizado Extremo |0.9456    | 10010            |
|Rede MLP                       |0.9887    |407050            |
|Rede Convolucional             |0.9949    |357610            |

- Como era esperado, o classificador linear apresentou o pior desempenho dentre os modelos, devido ao número reduzido de parâmetros ajustáveis e a propriedade de gerar apenas fronteiras de decisão lineares para separação das classes.

- Na sequência está a máquina de aprendizado extremo, cujo ganho de desempenho deve-se a aplicação de funções de ativação não-lineares nos dados de entrada, tornando o modelo capaz de gerar mapeamentos (e fronteiras de decisão) não-lineares e mais flexibilidade. 
- No entanto, o desempenho da ELM é inferior ao da rede MLP pois a flexibilidade alcançada pela ELM é menor, sendo consequência do menor número de parâmetros ajustáveis.
- Já as redes MLP e convolucional estudadas nesse EFC foram capazes de superar significativamente o desempenho dos modelos anteriores. No caso da MLP foi alcançado um desempenho de 98.87%, enquanto a rede convolucional atingiu 99.49%.
- Podemos dizer que o alto desempenho da rede MLP deve-se ao elevado nível de flexibilidade do modelo devido ao seu número elevado de parâmetros ajustáveis.
- Por outro lado, o desempenho alcançado pela rede convolucional deve-se às camadas convolucionais e suas propriedades, tais como:
  - A rede convolucional não requer a vetorização das imagens de entrada.
  - Leva em conta o caráter espacial das imagens.
  - Há uma redução significativa do número de parâmetros ajustáveis, pois as camadas convolucionais realizam compartilhamento de pesos (as duas camadas convolucionais juntas possuem apenas 19040 pesos sinápticos).
  - Maior capacidade de extração de atributos pelos filtros convolucionais.



